Install our package, then import it.

_this is here as an example, and for development, eventually this install step shouldn't be in the notebooks, only in the main readme_

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import warnings

In [2]:
%%bash
cd .. 
pip install . --upgrade

Processing /home/smb/Documents/code/detect_simpsons_paradox_dev
  Running setup.py bdist_wheel for detect-simpsons-paradox: started
  Running setup.py bdist_wheel for detect-simpsons-paradox: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-spda73j_/wheels/6f/33/e5/2e2b04658a9facaec056625f497ea9c4a21448cb987ce85d62
Successfully built detect-simpsons-paradox
  Found existing installation: detect-simpsons-paradox 0.2
    Uninstalling detect-simpsons-paradox-0.2:
      Successfully uninstalled detect-simpsons-paradox-0.2


You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import detect_simpsons_paradox as dsp
import fairsim as sp_dat 
#warnings.simplefilter(action='ignore', category=FutureWarning)

# Dataset Generation

In [4]:
# Use geometric construction of SP from GMM

r_clusters = -.6  # correlation coefficient of clusters
cluster_spread = .8 # pearson correlation of means
p_sp_clusters = .5 # portion of clusters with SP 
k = 5 # number of clusters
cluster_size = [2,3]
domain_range = [0, 20, 0, 20]
N = 200 # number of points
p_clusters = [1.0/k]*k

sp_df2 = sp_dat.geometric_2d_gmm_sp(r_clusters,cluster_size,cluster_spread,
                                    p_sp_clusters, domain_range,k,N,p_clusters)
#mixed_regression_sp(N,mu,cov,[.7,.3])

In [5]:
sp_df2.head()

,x1,x2,color
0,11.977708,9.551939,0
1,13.750940,13.735638,2
2,6.652990,4.315186,3
3,4.277728,9.848916,1
4,7.385700,4.078566,3


# Extract contiunous variables and categorical variables

Categorical attributes' types contain 'object' and 'int64'.
Continuous attributes' type contains float64.

In [7]:
df = dsp.get_subgroup_trends(sp_df2,['pearson_corr'])

  feat1 feat2  agg_trend    trend_type
0    x1    x2   0.777088  pearson_corr
  feat1 feat2  subgroup_trend group_feat  subgroup    trend_type
0    x1    x2       -0.609661      color         0  pearson_corr
1    x1    x2       -0.693888      color         1  pearson_corr
2    x1    x2       -0.423995      color         2  pearson_corr
3    x1    x2        0.463856      color         3  pearson_corr
4    x1    x2        0.458352      color         4  pearson_corr


In [8]:
df.head()

,feat1,feat2,subgroup_trend,group_feat,subgroup,trend_type,agg_trend
0,x1,x2,-0.609661,color,0,pearson_corr,0.777088
1,x1,x2,-0.693888,color,1,pearson_corr,0.777088
2,x1,x2,-0.423995,color,2,pearson_corr,0.777088
3,x1,x2,0.463856,color,3,pearson_corr,0.777088
4,x1,x2,0.458352,color,4,pearson_corr,0.777088


In [9]:
gb = sp_df2.groupby('color')
len(gb)

5

In [15]:
tui = np.triu_indices(2,k=1)
tui

(array([0]), array([1]))

# TODO Visualization (Hard code modify)

In [ ]:
# group by color
grouped_df_color = latent_df.groupby('color')['x1','x2','x3'].corr()
blue=grouped_df_color.loc['b']
red=grouped_df_color.loc['r']

# group by char
grouped_df_char = latent_df.groupby('char')['x1','x2','x3'].corr()
char_o=grouped_df_char.loc['o']
char_x=grouped_df_char.loc['x']

# create a 0 matrix for the irrelvant cells
s = (len(blue),len(blue))
zero = np.zeros(s)

# manipulate the subgroup matrix first after corr() method
color_group=np.vstack([np.hstack([blue, zero]), np.hstack([zero, red])])

# create a 0 matrix for the irrelvant cells
s = (len(all_corr),len(color_group))
zero1 = np.zeros(s)

# combine the correlation matrix of all of the data with subgroup by color
all_combined=np.vstack([np.hstack([all_corr, zero1]), np.hstack([zero1.transpose(), color_group])])

# Combine char group
s = (len(char_o),len(char_x))
zero = np.zeros(s)
char_group=np.vstack([np.hstack([char_o, zero]), np.hstack([zero, char_x])])
s = (len(all_combined),len(char_group))
zero1 = np.zeros(s)
all_combined=np.vstack([np.hstack([all_combined, zero1]), np.hstack([zero1.transpose(), char_group])])

__ why a heat map?__ 

In [ ]:
# Generate a heatmap for correlation matrix
cmap = mcolors.LinearSegmentedColormap.from_list('mycmap', ['red', 'white', 'blue'])
norm = plt.Normalize(-1,1)

labels =  ['x1','x2','x3','x1_b','x2_b','x3_b','x1_r','x2_r','x3_r','x1_o','x2_o','x3_o','x1_x','x2_x','x3_x']
plt.figure(figsize=(11, 9))

ax = sns.heatmap(all_combined, cmap=cmap, annot=True, fmt=".2f", xticklabels=labels, yticklabels=labels, linewidths=.5, norm=norm, center=0)

# Offset 0.00
for t in ax.texts: 
    if t.get_text() == '0.00':
        t.set_text("")
plt.show()